In [1]:
%matplotlib inline

from tqdm import tqdm
import numpy as np
import pandas as pd
import folium
import matplotlib as mpl
import matplotlib.cm as cm
from transliterate import translit
from geopy import distance

In [2]:
df = pd.read_csv('Rus_schools_final.csv', encoding='cp1251')
df.head()

,Unnamed: 0,name,struct,addr,lat,lon
0,0,"Муниципальное общеобразовательное учреждение ""...",(Муниципальное образовательное учреждение),"420087, Республика Татарстан, г. Казань, ул.Ри...",55.76370,49.18172
1,1,"Муниципальная образовательная школа-интернат ""...",(Муниципальное образовательное учреждение),"420103, Республика Татарстан, г. Казань, ул.Че...",55.82440,49.12312
2,2,Муниципальное учреждение образования для детей...,(Муниципальное образовательное учреждение),"420100, Республика Татарстан, г. Казань, ул. Ю...",55.74462,49.20560
3,3,"Муниципальное общеобразовательное учреждение ""...",(Муниципальное образовательное учреждение),"420011, Республика Татарстан, г. Казань, Ферма-2",55.71716,49.16222
4,4,Специальное (коррекционное)образовательное учр...,(Государственное образовательное учреждение),"420036, Республика Татарстан, г. Казань, ул.Ти...",55.84911,49.07660


In [3]:
center = {'lat': 55.756204, 'lon': 37.616675}

In [4]:
df = df[(df['addr'].str.contains('Москва'))]
my_school = df[(df['name'].str.contains(' 738 '))]

df = df[~(df['name'].str.contains(' 738 '))]

In [5]:
map_folium = folium.Map(location=(center['lat'], center['lon']),
                        #tiles='Stamen Terrain',
                        zoom_start=11)

In [6]:
norm = mpl.colors.Normalize(vmin=0, vmax=30000)
cmap = cm.magma
colormap= cm.ScalarMappable(norm=norm, cmap=cmap)

for ind, row in df.iterrows():
    
    dist = distance.distance((row['lat'], row['lon']), 
                             (my_school['lat'].values[0], my_school['lon'].values[0])
                            ).meters
    
    folium.CircleMarker([row['lat'], row['lon']],
                        radius=3,
                        weight=1,
                        popup='{}'.format(translit(row['name'], reversed=True)),
                        color=mpl.colors.rgb2hex((colormap.to_rgba(dist))),
                        fill_color=mpl.colors.rgb2hex((colormap.to_rgba(dist))),
                        fill=True, 
                        fill_opacity=0.9
                       ).add_to(map_folium)
    

In [7]:
folium.CircleMarker([my_school['lat'], my_school['lon']],
                    radius=12,
                    weight=1,
                    popup='{}'.format(translit(my_school['name'].values[0], reversed=True)),
                    color='green',
                    fill_color='#B3BABA',
                    fill=True, 
                    fill_opacity=1
                   ).add_to(map_folium)


folium.Marker(
    location=[my_school['lat'], my_school['lon']],
    popup='This is my school',
    icon=folium.Icon(color='green', icon=None)
).add_to(map_folium)

map_folium